In [10]:
import pandas as pd
import numpy as np
#import ast
import seaborn as sns

from matplotlib.pyplot import figure


import matplotlib.pyplot as plt
from matplotlib import cm
from collections import Counter


#from datetime import date
#import calendar

#from scipy.stats import chi2_contingency
#from pandas.plotting import scatter_matrix



#Ahora scalamos la data para encontrar los segmentos
from sklearn import preprocessing 
from sklearn.cluster import KMeans

#Para balancear la data
#from imblearn.over_sampling import SMOTE


# Para modelar:


from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


from statsmodels.discrete.discrete_model import Logit
import statsmodels.api as sm
from sklearn import metrics

import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score,f1_score

#Para interpretar el sentido de las variables en ML
import shap
#Para guardar
import joblib


In [11]:
df = pd.read_csv('ds_challenge_2021.csv',encoding="UTF-8")
df = df.reset_index(drop=True)

In [12]:
#df.head()

In [13]:
# Modificación del feature 'dispositivo'
dispositivo = df["dispositivo"].apply(lambda x : dict(eval(x))).apply(pd.Series)
df.drop(columns = ['dispositivo'], axis = 1, inplace = True)

# Data Frame con todas las features
df = pd.concat([df,dispositivo],axis = 1)

## Variables que se necesitan para los modelos y segmentacion

In [14]:
var_in = [
'ID_USER',
'monto',
'status_txn',
'os',

'linea_tc',
'interes_tc',
'device_score',
 
# Adicional Modelo 1     
'dcto',
'cashback',
'hora',
'tipo_tc',
'is_prime'    
    
 ]

df = df[var_in]
df.head()

,ID_USER,monto,status_txn,os,linea_tc,interes_tc,device_score,dcto,cashback,hora,tipo_tc,is_prime
0,0,608.345634,Aceptada,ANDROID,71000,51,3,60.834563,5.475111,20,Física,False
1,0,88.719243,Aceptada,ANDROID,71000,51,1,0.000000,1.774385,7,Virtual,False
2,1,790.037653,En proceso,.,94000,56,1,0.000000,15.800753,11,Virtual,False
3,1,767.005387,Aceptada,WEB,94000,56,3,0.000000,7.670054,11,Física,False
4,1,254.171053,Aceptada,%%,94000,56,3,0.000000,2.541711,10,Física,False


In [15]:
#Creando las otras variables

df["status_txn_Rechazada"] = np.where(df["status_txn"]=='Rechazada',1,0)
df["os_ANDROID"] = np.where(df["os"]=='ANDROID',1,0)
df["os_WEB"] = np.where(df["os"]=='WEB',1,0)

df["FLG_prime"] = np.where(df["is_prime"]==False,0,1)
df["tipo_tc_Física"] = np.where(df["tipo_tc"]=='Física',1,0)

df.drop(columns = ['status_txn','os','tipo_tc'], axis = 1, inplace = True)
df.head()

,ID_USER,monto,linea_tc,interes_tc,device_score,dcto,cashback,hora,is_prime,status_txn_Rechazada,os_ANDROID,os_WEB,FLG_prime,tipo_tc_Física
0,0,608.345634,71000,51,3,60.834563,5.475111,20,False,0,1,0,0,1
1,0,88.719243,71000,51,1,0.000000,1.774385,7,False,0,1,0,0,0
2,1,790.037653,94000,56,1,0.000000,15.800753,11,False,0,0,0,0,0
3,1,767.005387,94000,56,3,0.000000,7.670054,11,False,0,0,1,0,1
4,1,254.171053,94000,56,3,0.000000,2.541711,10,False,0,0,0,0,1


## Generar el modelo 1  Regresión logística a nivel trxs

In [16]:
# Cargando variables
df_trx = df.copy()
var_model1 = joblib.load('var_model_trxs.pkl')
# Cargando modelo
model1 = joblib.load('reg_log_trxs.pkl')
scaler = joblib.load('scale_reg_log_trxs.pkl')

#%% Se normalizan los datos con MinMax()
#min_max_scaler = preprocessing.MinMaxScaler() 
#df_escalado = min_max_scaler.fit_transform(df_trx[var_model1])

#scaler = preprocessing.MinMaxScaler() 
a = df_trx[var_model1].columns
X_scaled = scaler.fit(df_trx[var_model1]).transform(df_trx[var_model1])

X_scaled = pd.DataFrame(X_scaled, columns=a)
X_scaled = X_scaled.reset_index(drop=True)
#df_total_scale = pd.concat((df_cli_clus2,X_scaled),axis=1)
X_scaled = X_scaled.reset_index(drop=True)
X_scaled.head()

#df_cliente[var_model1].head()
pred_model1 = model1.predict(X_scaled)


In [17]:
df_trx[var_model1].head()

,monto,hora,linea_tc,is_prime,dcto,cashback,tipo_tc_Física,os_ANDROID,os_WEB
0,608.345634,20,71000,False,60.834563,5.475111,1,1,0
1,88.719243,7,71000,False,0.000000,1.774385,0,1,0
2,790.037653,11,94000,False,0.000000,15.800753,0,0,0
3,767.005387,11,94000,False,0.000000,7.670054,1,0,1
4,254.171053,10,94000,False,0.000000,2.541711,1,0,0


In [19]:
X_scaled.head()

,monto,hora,linea_tc,is_prime,dcto,cashback,tipo_tc_Física,os_ANDROID,os_WEB
0,0.608389,0.863636,0.621622,0.0,0.305155,0.273866,1.0,1.0,0.0
1,0.088711,0.272727,0.621622,0.0,0.000000,0.088750,0.0,1.0,0.0
2,0.790099,0.454545,0.932432,0.0,0.000000,0.790369,0.0,0.0,0.0
3,0.767064,0.454545,0.932432,0.0,0.000000,0.383660,1.0,0.0,1.0
4,0.254179,0.409091,0.932432,0.0,0.000000,0.127133,1.0,0.0,0.0


In [18]:
# Df Final:
df_result_trx = pd.DataFrame({'ID_USER': df_trx['ID_USER'], 
                                                
                          'Probabilidad':pred_model1})
df_result_trx["fraude_pred"] = np.where(df_result_trx["Probabilidad"]>0.515,1,0)                                               
df_result_trx.head()     
#Exportar data

,ID_USER,Probabilidad,fraude_pred
0,0,0.498459,0
1,0,0.519788,1
2,1,0.496858,0
3,1,0.465467,0
4,1,0.511563,0


## Hacer la data a nivel cliente

In [120]:
# Cargando variables
df_cliente = df.copy()

In [121]:
# 1. Basado en criterio tomamos variables para haer un resumen de la suma total a nivel usuario
df_num = df_cliente.groupby(['ID_USER'])[['monto','is_prime',  'tipo_tc_Física', 'status_txn_Rechazada', 'os_ANDROID',
        'os_WEB']].sum().reset_index()
df_num.head()

,ID_USER,monto,is_prime,tipo_tc_Física,status_txn_Rechazada,os_ANDROID,os_WEB
0,0,697.064876,0,1,0,2,0
1,1,3416.796066,0,5,1,0,1
2,2,2789.893416,0,6,0,3,3
3,3,477.972148,0,2,0,2,0
4,4,277.176524,0,1,0,0,1


In [122]:
# 2. Nº trxs por ID_USER
# No va en el cluster pero si en el modelo final
df_trxs = df_cliente.groupby(['ID_USER'])[['ID_USER']].count().rename(columns={"ID_USER":"Trxs"}).reset_index()
df_trxs.head()

,ID_USER,Trxs
0,0,2
1,1,7
2,2,7
3,3,2
4,4,1


In [123]:
# 6. Promedio de las siguientes variables
df_prom = df_cliente.groupby(['ID_USER'])[['linea_tc', 'interes_tc','device_score']].mean().reset_index()
df_prom

,ID_USER,linea_tc,interes_tc,device_score
0,0,71000.0,51.0,2.000000
1,1,94000.0,56.0,2.428571
2,2,55000.0,57.0,2.714286
3,3,62000.0,34.0,2.500000
4,4,83000.0,37.0,2.000000
...,...,...,...,...
3995,3995,79000.0,37.0,2.571429
3996,3996,25000.0,57.0,3.333333
3997,3997,45000.0,52.0,3.000000
3998,3998,28000.0,39.0,2.333333


In [124]:
#Flag Fraude y Flag Prime
df_num["FLG_prime"] = np.where(df_num["is_prime"]>=1,1,0)
df_num.drop(columns = 'is_prime', axis = 1, inplace = True)
df_num.head()

,ID_USER,monto,tipo_tc_Física,status_txn_Rechazada,os_ANDROID,os_WEB,FLG_prime
0,0,697.064876,1,0,2,0,0
1,1,3416.796066,5,1,0,1,0
2,2,2789.893416,6,0,3,3,0
3,3,477.972148,2,0,2,0,0
4,4,277.176524,1,0,0,1,0


In [125]:
#Unir todas las bases
df_cli_clus = pd.merge(df_num,df_trxs,left_on=['ID_USER'],right_on=['ID_USER'],how='left')
df_cli_clus = pd.merge(df_cli_clus,df_prom,left_on=['ID_USER'],right_on=['ID_USER'],how='left')
print(df_cli_clus.shape)
df_cli_clus.head()

(4000, 11)


,ID_USER,monto,tipo_tc_Física,status_txn_Rechazada,os_ANDROID,os_WEB,FLG_prime,Trxs,linea_tc,interes_tc,device_score
0,0,697.064876,1,0,2,0,0,2,71000.0,51.0,2.000000
1,1,3416.796066,5,1,0,1,0,7,94000.0,56.0,2.428571
2,2,2789.893416,6,0,3,3,0,7,55000.0,57.0,2.714286
3,3,477.972148,2,0,2,0,0,2,62000.0,34.0,2.500000
4,4,277.176524,1,0,0,1,0,1,83000.0,37.0,2.000000


## Generar cluster

In [126]:
var_clust = joblib.load('var_cluster.pkl')
# Cargando modelo
clust = joblib.load('segm_kmeans.pkl')
scaler = joblib.load('scale_segm_kmeans.pkl')


#%% Se normalizan los datos con MinMax()

#scaler = preprocessing.MinMaxScaler() 
a = df_cli_clus[var_clust].columns
X_scaled = scaler.fit(df_cli_clus[var_clust]).transform(df_cli_clus[var_clust])

X_scaled = pd.DataFrame(X_scaled, columns=a)
X_scaled = X_scaled.reset_index(drop=True)
#df_total_scale = pd.concat((df_cli_clus2,X_scaled),axis=1)
df_total_scale = X_scaled.reset_index(drop=True)
print(df_total_scale.shape)
print(df_cli_clus[var_clust].shape)
df_total_scale.head()

(4000, 8)
(4000, 8)


,monto,status_txn_Rechazada,os_ANDROID,os_WEB,FLG_prime,linea_tc,interes_tc,device_score
0,0.021219,0.000000,0.133333,0.000000,0.0,0.621622,0.59375,0.250000
1,0.104252,0.111111,0.000000,0.058824,0.0,0.932432,0.75000,0.357143
2,0.085113,0.000000,0.200000,0.176471,0.0,0.405405,0.78125,0.428571
3,0.014530,0.000000,0.133333,0.000000,0.0,0.500000,0.06250,0.375000
4,0.008400,0.000000,0.000000,0.058824,0.0,0.783784,0.15625,0.250000


In [127]:
#Obtenemos el cluster y lo colocamos en nuestra data
cluster_labels = clust.fit_predict(df_total_scale)
cluster_labels
df_cli_clus['Cluster'] = cluster_labels

clust_map = {
    0:'BV_RI',
    1:'MV_RA',
    2:'BV_RB',
    3:'AV_RMA'
}

df_cli_clus.Cluster = df_cli_clus.Cluster.map(clust_map)


In [128]:
df_cli_clus.Cluster.value_counts(normalize='index')

MV_RA     0.32500
AV_RMA    0.27025
BV_RI     0.26925
BV_RB     0.13550
Name: Cluster, dtype: float64

In [129]:
#Cluster
df_cli_clus_fin = df_cli_clus[['ID_USER','Cluster']]
df_cli_clus_fin.head()
#Exportar

,ID_USER,Cluster
0,0,BV_RI
1,1,BV_RI
2,2,AV_RMA
3,3,MV_RA
4,4,MV_RA


## Generar el modelo LGBM nivel cliente

In [139]:
# Cargando variables
df_cliente_model = df_cli_clus.copy()

clust_map = {
    'BV_RI':3,
    'MV_RA':2,
    'BV_RB':4,
    'AV_RMA':1
}

df_cliente_model.Cluster = df_cliente_model.Cluster.map(clust_map)
df_cliente_model.head()

,ID_USER,monto,tipo_tc_Física,status_txn_Rechazada,os_ANDROID,os_WEB,FLG_prime,Trxs,linea_tc,interes_tc,device_score,Cluster
0,0,697.064876,1,0,2,0,0,2,71000.0,51.0,2.000000,3
1,1,3416.796066,5,1,0,1,0,7,94000.0,56.0,2.428571,3
2,2,2789.893416,6,0,3,3,0,7,55000.0,57.0,2.714286,1
3,3,477.972148,2,0,2,0,0,2,62000.0,34.0,2.500000,2
4,4,277.176524,1,0,0,1,0,1,83000.0,37.0,2.000000,2


In [151]:
var_lgbm = joblib.load('var_model_cliente_lgbm.pkl')
# Cargando modelo
lgbm_model = joblib.load('lgbm_cliente.pkl')
scaler = joblib.load('scale_lgbm_cliente.pkl')




#%% Se normalizan los datos con MinMax()

#scaler = preprocessing.MinMaxScaler() 
a = df_cliente_model[var_lgbm].columns
X_scaled = scaler.fit(df_cliente_model[var_lgbm]).transform(df_cliente_model[var_lgbm])

X_scaled = pd.DataFrame(X_scaled, columns=a)
X_scaled = X_scaled.reset_index(drop=True)
#df_total_scale = pd.concat((df_cli_clus2,X_scaled),axis=1)
X_scaled = X_scaled.reset_index(drop=True)
X_scaled.head()

#df_cliente[var_model1].head()
pred_model_lgbm = lgbm_model.predict_proba(X_scaled[var_lgbm])[:,1]
#pred_model_lgbm


In [152]:
# Df Final:
df_result_cli = pd.DataFrame({'ID_USER': df_cli_clus['ID_USER'], 
                                                
                          'Probabilidad':pred_model_lgbm})
df_result_cli["fraude_pred"] = np.where(df_result_cli["Probabilidad"]>0.5,1,0)                                               
df_result_cli.head()   

,ID_USER,Probabilidad,fraude_pred
0,0,0.390660,0
1,1,0.498941,0
2,2,0.514547,1
3,3,0.404966,0
4,4,0.363781,0


In [155]:
df_result_cli["SegmentoFraude"] = np.where(df_result_cli["Probabilidad"]<=0.4,"1. Fraude Alto",
                                             np.where(df_result_cli["Probabilidad"]<=0.502,"2. Fraude Medio","3. Fraude Bajo"))
df_result_cli.head()                                               

,ID_USER,Probabilidad,fraude_pred,SegmentoFraude
0,0,0.390660,0,1. Fraude Alto
1,1,0.498941,0,2. Fraude Medio
2,2,0.514547,1,3. Fraude Bajo
3,3,0.404966,0,2. Fraude Medio
4,4,0.363781,0,1. Fraude Alto


In [156]:
df_result_cli.SegmentoFraude.value_counts()

3. Fraude Bajo     1338
1. Fraude Alto     1337
2. Fraude Medio    1325
Name: SegmentoFraude, dtype: int64

In [ ]:
#Exportar data

In [90]:
var_lgbm

Index(['monto', 'tipo_tc_Física', 'status_txn_Rechazada', 'os_ANDROID',
       'os_WEB', 'FLG_prime', 'Trxs', 'linea_tc', 'interes_tc', 'device_score',
       'Cluster'],
      dtype='object')

In [92]:
lgbm_model.summary

<bound method BinaryResults.summary of <statsmodels.discrete.discrete_model.LogitResults object at 0x00000141F0128BE0>>

In [ ]:
df.drop(columns = ['dispositivo'], axis = 1, inplace = True)

In [ ]:
'status_txn_Rechazada'
'os_ANDROID',
'os_WEB',
'FLG_prime',
'tipo_tc_Física',


In [ ]:
# Armar la data para el modelo 1
var_in = [
# Para cluster 
'monto',
'status_txn_Rechazada',
'os_ANDROID',
'os_WEB',
'FLG_prime',
'linea_tc',
'interes_tc',
'device_score',
 
# Adicional Modelo 1     
'dcto',
'cashback',
'hora',
'tipo_tc_Física',
'is_prime',
    
#Para modelo nivel cliente - LGBM
'Trxs',
'Cluster'   
         ]

In [ ]:
var_clus = ['monto',
'status_txn_Rechazada',
'os_ANDROID',
'os_WEB',
'FLG_prime',
'linea_tc',
'interes_tc',
'device_score']

In [ ]:
dcto
0.686665
monto
0.317923
cashback
0.310043
hora
0.216738
linea_tc
0.112549
os_WEB
0.110943
tipo_tc_Física
0.103934
os_ANDROID
0.090733
is_prime
0.04246

In [ ]:
'monto', 'tipo_tc_Física', 'status_txn_Rechazada', 'os_ANDROID',
       'os_WEB', 'FLG_prime', 'Trxs', 'linea_tc', 'interes_tc', 'device_score',
       'Cluster'

In [4]:
# Armar la data nivel cliente y hacer la segmentacion 

In [ ]:
# Armar la data para el modelo LGBM cliente